In [9]:
import cv2
import numpy as np
import keras
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import adam
from keras.preprocessing import image

In [10]:
model=Sequential()

#first layer
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )

#second layer
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )

#third layer
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )

#fourth layer
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
training_set = train_datagen.flow_from_directory(
        r'C:\Users\Avinash\Desktop\FaceMask\train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        r'C:\Users\Avinash\Desktop\FaceMask\test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [5]:
model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,
        )

Epoch 1/10
83/83 [==============================] - 23s 272ms/step - loss: 0.3128 - accuracy: 0.8607 - val_loss: 0.0148 - val_accuracy: 0.9381
Epoch 2/10
83/83 [==============================] - 26s 311ms/step - loss: 0.1455 - accuracy: 0.9521 - val_loss: 0.9478 - val_accuracy: 0.9072
Epoch 3/10
83/83 [==============================] - 27s 330ms/step - loss: 0.1373 - accuracy: 0.9581 - val_loss: 1.4556 - val_accuracy: 0.9536
Epoch 4/10
83/83 [==============================] - 34s 408ms/step - loss: 0.0878 - accuracy: 0.9726 - val_loss: 0.0571 - val_accuracy: 0.9742
Epoch 5/10
83/83 [==============================] - 33s 399ms/step - loss: 0.0903 - accuracy: 0.9680 - val_loss: 0.0011 - val_accuracy: 0.9794
Epoch 6/10
83/83 [==============================] - 34s 410ms/step - loss: 0.1502 - accuracy: 0.9536 - val_loss: 9.4973e-04 - val_accuracy: 0.9381
Epoch 7/10
83/83 [==============================] - 32s 388ms/step - loss: 0.1040 - accuracy: 0.9627 - val_loss: 0.0156 - val_accuracy: 0.

In [11]:
model.save('facemaskmodel.h5')

In [12]:
my_model=load_model('facemaskmodel.h5')

cap=cv2.VideoCapture(0)

while cap.isOpened():
    ret,img=cap.read()
    face_img=cv2.resize(img,(150,150))
    face_img=face_img/255.0
    face_img=np.reshape(face_img,(150,150,3))
    face_img=np.expand_dims(face_img,axis=0)
    pred=my_model.predict_classes(face_img)
    if pred[0][0]==0:
        cv2.putText(img,'NO MASK',(250,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),4)
    else:
        cv2.putText(img,'MASK',(250,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4)
        
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
   